# Getting Started

This tutorial will help you install the ESGF End-user API and demonstrate some of the capabilites using ESGF's WPS server on [aims2](https://aims2.llnl.gov).

## Step 1

Install [conda](https://conda.io/docs/install/quick.html) for your environment.

## Step 2

```
conda install -c uvcdat esgf-compute-api
```

## Step 3

Here you'll need to register for an account and retrieve your API key.

Visit [aims2](https://aims2.llnl.gov/wps/home/create) to create your account. Once you've created your account navigate to the [profile](https://aims2.llnl.gov/wps/home/user) page and authenticate through MyProxyClient or OAuth2. Copy you API key as this will be needed for the next step.

## Step 4

Next we'll create a basic python script to access the WPS server and go over some sample usage.

Create a python script with the following:

In [ ]:
api_key = 'YOUR KEY'

import cwt

wps = cwt.WPS('https://aims2.llnl.gov/wps', api_key=api_key)

## List the available processes
This will print out a list of the available processes.

In [ ]:
for p in wps.processes():
    print p.identifier

## Choose a process to execute
Let's choose which process we'll be executing.

In [ ]:
proc = wps.get_process('CDAT.subset')

## Configure our process with inputs and some constraints
Here we define our input variables and the domain we're interested in.

In [ ]:
tas = cwt.Variable('http://aims3.llnl.gov/thredds/dodsC/cmip5_css02_data/cmip5/output1/CMCC/CMCC-CM/decadal2005/mon/atmos/Amon/r1i2p1/tas/1/tas_Amon_CMCC-CM_decadal2005_r1i2p1_200511-201512.nc', 'tas')

d0 = cwt.Domain([
    cwt.Dimension('latitude', -90, 0),
    cwt.Dimension('longitude', 180, 360),
    cwt.Dimension('time', 100, 300, cwt.INDICES),
])

## Let's execute the process
Here we'll execute the process on the server. We're using the inputs and domain we defined above and applying regridding to grid T21.

In [ ]:
wps.execute(proc, inputs=[tas], domain=d0, gridder=cwt.T21)

## Now, let's check the status

In [ ]:
import time

while proc.processing:
    print proc.status
    
    time.sleep(1)
    
print proc.status

print proc.output.uri
print proc.output.var_name

## Let's explore the output

In [ ]:
import cdms2

f = cdms2.open(proc.output.uri)

tas = f['tas']

print tas.shape